In [3]:
import os, re, math, json, shutil, pprint
import IPython.display as display
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib
import matplotlib.pyplot as plt
import pathlib
from skimage import io
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.callbacks import History
from sklearn.preprocessing import LabelEncoder
print(tf.__version__)

2.8.2


In [ ]:
from os import listdir
from os.path import isfile, join

images = []
labels = []
dataset_path = './MLBD-dataset-IML-2022-Anglo-Saxion-Runes/'
for file_name in listdir(dataset_path):
    full_path = join(dataset_path, file_name)
    if isfile(full_path):
        if re.match(r".*\.png$", full_path):
            image_current = io.imread(full_path, as_gray=True)
            image_label = file_name.split('_')[0].lower()
            # append to the array
            images.append(image_current)
            labels.append(image_label)

Basic method to define the structure of the convolutional neural network

Our structure consists of the following:

1. Input layer with the shape of the image (128, 128, 1)
2. (layers) that would be defined later
3. Flatten layer to flatten the output of the previous layers
4. Dense layer with 512 neurons and relu activation function
5. Desnse layer for our output with 11 neurons (which represents the runes) and softmax activation function

As you can observe, we use the adam optimizer and the categorical crossentropy loss function and using the metrics for the accuracy.

In [1]:
def create_model(layers=[], kernel_size =(3, 3), debug=False):
    model = tf.keras.models.Sequential()
    # Define the input layer 1 channel
    model.add(tf.keras.layers.Input(shape=(128, 128, 1))) 
    
    # Adding convolutional layers
    for layer in layers:
        model.add(layer)

    # Flatten the output from the convolutional layers
    model.add(tf.keras.layers.Flatten())

    # Define the fully connected layer
    model.add(tf.keras.layers.Dense(512, activation='relu'))

    # Output layer with 11 neurons as we have 11 runes to classify
    # which is using the softmax activation function
    model.add(tf.keras.layers.Dense(11, activation='softmax'))

    if (debug):
        model.summary()
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

We're also using the early stopping callback to stop the training if the validation loss is not decreasing for 5 epochs.

source: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

In [ ]:
def early_stopping():
    return tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Stores the model and the history of the training

In [2]:
models_and_history = []

# Experiment 1

# Experiment 2

# Experiment 3

# Experiment 4

# Experiment 5

# Experiment 6

# Conclusion

#todo